# Video Preprocessing

Description

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import json
import gc
from tqdm.notebook import tqdm

In [3]:
from preprocess.video_analyzer import VideoAnalyzer, analyze_none_landmarks
from preprocess.preprocessor import Preprocessor

In [4]:
timestamp = "04242025"
motion_version = "versionB"
pose_version = "versionB"
preprocessing_version = "v4"
path_to_root = "/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/"

In [5]:
metadata = pd.read_csv(os.path.join(
    path_to_root,
    "data",
    "raw",
    "combined",
    "target_dataset_video_metadata.csv"
    ))

In [ ]:
for i, metadata_row in metadata[:].iterrows():
    print(f"\rProcessing video {i+1} of {len(metadata)}: {metadata_row.filename}", end="")
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
    analyzer = VideoAnalyzer(
        metadata_row,
        timestamp,
        path_to_root,
        verbose=False,
        motion_detection_version=motion_version,
        pose_detection_version=pose_version
    )
    pose_data = analyzer.pose_detect()
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    pose_result = analyzer.pose_analyze()
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
    motion_data = analyzer.motion_detect()
    motion_result = analyzer.motion_analyze()

    analyzer.save_analysis_info()

In [11]:
for i, metadata_row in tqdm(metadata[:].iterrows(), total=len(metadata)):
    gc.collect()
    
    with open(os.path.join(
        path_to_root, 
        "data", 
        "interim", 
        "Analysis",
        f"{timestamp}_motion{motion_version}_pose{pose_version}", 
        metadata_row["filename"].split(".")[0] + "_analysis_info.json"
        )) as f:
        analysis_info = json.load(f)
    
    preprocessing_params = {
        "face_width_aim": 0.155,
        "shoulders_width_aim": 0.35,
        "face_midpoint_to_shoulders_height_aim": 0.275,
        "shoulders_y_aim": 0.52,
        "use_statistic": "mean",
        "use_stationary_frames": True,
        "skip_stationary_frames": False,
        "start_frame": analysis_info['motion_analysis']['start_frame'],
        "end_frame": analysis_info['motion_analysis']['end_frame'],
    }

    preprocessor = Preprocessor(
            metadata_row,
            preprocessing_params,
            path_to_root,
            preprocess_version=preprocessing_version,
            verbose=False,
            save_intermediate=True,
        )

    preprocessor.preprocess_landmarks()
    # preprocessor.preprocess_video()
    

        
    # Force garbage collection after each video
    gc.collect()

  0%|          | 0/150 [00:00<?, ?it/s]